In [1]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
import random

# Web Scraping: Selenium

In [2]:
options = Options()
ua = UserAgent()
user_agent = ua.random
print(user_agent)
options.add_argument(user_agent)
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36


## Collect Company name- Method 1 from company review page

In [3]:
query = "Data"
indeed_search = "https://www.indeed.com/companies/search?q="
indeed_query = indeed_search + query + "&l=&from=discovery-cmp-search"
indeed_query


'https://www.indeed.com/companies/search?q=Data&l=&from=discovery-cmp-search'

In [4]:
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
options = Options()
ua = UserAgent()
user_agent = ua.random
print(user_agent)
options.add_argument(f'user-agent={user_agent}')

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1664.3 Safari/537.36


In [5]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
driver.get(indeed_query)

<ipython-input-5-58066457306e>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [7]:
cmp_n=soup.find_all('a',{"itemprop":"name","class":"cmp-CompanyWidget-name"})

#for cmp in cmp_n[1:]:
#    print(cmp.text)

In [8]:
job_title=["Data+Analyst","Data+Engineers","Database+Administrator",
           "Machine+Learning+Engineer","Data+Scientist","Data+Architect",
           "Statistician""Business+Analyst","Data","Analytics+Manager",
           "Big+Data+Engineer","Database+Developer","Statistician",
           "Big+Data+Software+Developer","Business+Analyst",
           "Business+Intelligence+Analyst","Analyst","Analysis","Python","SQL",
           "BI+Specialist","Analytics+Manager","Machine+Learning+Scientist","Big+Data"]

In [9]:
def web_scrapping_cmp(title):
    
    headers = ['company']
    query = title
    indeed_search = "https://www.indeed.com/companies/search?q="
    indeed_query = indeed_search + query + "&l=&from=discovery-cmp-search"

    options = Options()
    ua = UserAgent()
    user_agent = ua.random
    options.add_argument(f'user-agent={user_agent}')

    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(indeed_query)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    cmp_n=soup.find_all('a',{"itemprop":"name","class":"cmp-CompanyWidget-name"})
    company_name=[]
    for cmp in cmp_n[1:]:
        company_name.append(cmp.text)
    headers = ['company']*len(company_name)
    driver.quit()
    return dict(zip(headers, [company_name]))

In [10]:
company_list = []

for link in job_title:
    company_list.append(web_scrapping_cmp(link))

<ipython-input-9-9f2bb662e799>:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


KeyboardInterrupt: 

In [11]:
print(len(company_list)) #23
#company_list

4


In [12]:
Indeed_Company=pd.DataFrame(company_list[0])
for i in range(1,len(company_list)):
    try:
        Indeed_Company=pd.concat([Indeed_Company,pd.DataFrame(company_list[i])])
    except Exception:
        pass

In [13]:
#driver.quit()
len(Indeed_Company.company.unique())
cmps=pd.DataFrame(Indeed_Company.company.unique())
cmps[0]

0                                 Pearson
1                                Deloitte
2                                  PayPal
3      Forfeiture Support Associates, LLC
4                                 Centene
                      ...                
124                               Dascena
125                              Battelle
126                    Seagate Technology
127                                Square
128                      Johnson Controls
Name: 0, Length: 129, dtype: object

## Collect Company name- Method 2 from job searching page

In [304]:
query_1 = "data-scientist"
indeed_search = "https://www.indeed.com/q-"
indeed_query_1 = indeed_search + query_1 + "-jobs.html"
indeed_query_1


'https://www.indeed.com/q-data-scientist-jobs.html'

In [14]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
driver.get(indeed_query_1)
next_page=driver.find_element_by_class_name('pn')
next_page.click()
#next_page=driver.find_element_by_class_name('pn')
#next_page.click()

next_page=driver.find_elements_by_css_selector('span.np')
next_page[1].click()
#<path d="M10 6L8.59 7.41 13.17 12l-4.58 4.59L10 18l6-6-6-6z" fill="#2D2D2D"></path>

<ipython-input-14-44809da2cbbe>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


NameError: name 'indeed_query_1' is not defined

In [293]:
data_title=["data-scientist","data-analyst","Data-Engineers","Data-Architect","Database-Developer"
            ,"Machine-Learning-Scientist","Machine-Learning-Engineer"]

#Indeed_Company=[]
for title in data_title:
    query_1 = title
    indeed_search = "https://www.indeed.com/q-"
    indeed_query_1 = indeed_search + query_1 + "-jobs.html"
    driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
    driver.get(indeed_query_1)
    
    for i in range(35):
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        comp_name=soup.find_all('a', {"data-tn-element":"companyName"})

        for cmp in comp_name:
            Indeed_Company.append(cmp.text.replace("\n",""))
            
        time.sleep(2)
        
        if i==0:
            next_page=driver.find_element_by_class_name('np')
            next_page.click()
        else:
            try:
                next_page=driver.find_elements_by_css_selector('span.np')
                next_page[1].click()
            except Exception:
                pass
    driver.quit()
    time.sleep(300)

print(len(Indeed_Company))


<ipython-input-293-123d17b6c614>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


2295


In [303]:
#driver.quit()
print(len(pd.DataFrame(Indeed_Company)[0].unique()))
new_company_list= pd.concat([cmps,pd.DataFrame(Indeed_Company)])
new_company_list=pd.DataFrame(new_company_list[0].unique())
new_company_list.to_csv('new_company_list.csv', index=False)

1251


### 1. Salary page

In [307]:
query1 = "Paypal/salaries"
indeed_search = "https://www.indeed.com/cmp/"
indeed_query1 = indeed_search + query1 +"?job_category=math"
indeed_query1

'https://www.indeed.com/cmp/Paypal/salaries?job_category=math'

In [308]:
from fake_useragent import UserAgent
from selenium.webdriver.chrome.options import Options
options = Options()
ua = UserAgent()
user_agent = ua.random
options.add_argument(f'user-agent={user_agent}')

##### First we launch the indeed page through our ChromeDrive. 
(A new browser should pop up. To continue using Selenium, keep this window open!)

In [309]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
driver.get(indeed_query1)

<ipython-input-309-e7bce6110ab5>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


In [310]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

#### Get Title/Salary/hourly or year

In [311]:
a=soup.find_all('span')

for a1 in a:
    try:
        a2=a1.text.split(" ")
        if a2[1]=='per':
            print(a2[0], a2[1]+" "+a2[2])
    except Exception:
        pass


$174,553 per year
$201,747 per year


In [312]:
soup.find_all('div', class_="cmp-SalarySummaryTitle")[0].find('a').text


'Data Scientist'

In [199]:
salary=soup.find_all('span')
title=soup.find_all('div', class_="cmp-SalarySummaryTitle")

i=0
for salary1 in salary:
    try:
        salary2=salary1.text.split(" ")
        if salary2[1]=='per':
            print(salary2[0], salary2[1]+" "+salary2[2])
            print(title[i].find('a').text)
            i+=1
    except Exception:
        pass

print(i)


$174,553 per year
Data Scientist
$201,747 per year
Lead Data Scientist
2


#### Salary Satisfication rate %

In [313]:
#salary_sat=soup.find('div',class_="cmp-SalarySatisfactionSidebarWidgetPieChart-inside").text
salary_sat=soup.find('div',class_="cmp-SalarySatisfactionSidebarWidgetPieChart-inside").text
salary_sat=int(salary_sat.replace("%",""))
salary_sat

67

In [314]:
#### Company Rating value & benefit rate

In [318]:
cmp_rating=float(soup.find('span', {"aria-hidden":"true", "class":"cmp-CompactHeaderCompanyRatings-value"}).text)
cmp_rating                 

3.9

In [325]:
query1 = "Paypal"
indeed_search = "https://www.indeed.com/cmp/"
indeed_query1 = indeed_search + query1

#driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
#driver.get(indeed_query1)
#soup = BeautifulSoup(driver.page_source, 'html.parser')

benefit_rating=float(soup.find_all('span', {"aria-hidden":"true","class":"css-3xwreb e1wnkr790"})[1].text)
print(benefit_rating)



4.0


#### 2. Company home page

In [161]:
query_2 = "Paypal"
indeed_search_2 = "https://www.indeed.com/cmp/"
indeed_query_2 = indeed_search_2 + query_2 + "/about?from=acme-wonder"
print(indeed_query_2)
driver.get(indeed_query_2)

https://www.indeed.com/cmp/Paypal/about?from=acme-wonder


In [162]:
soup_2 = BeautifulSoup(driver.page_source, 'html.parser')


#### Get Headquarters/Revenue/company size/ Industry

In [164]:
Headquarters_State=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                   "data-testid":"headquarters"}).text.split(',')[1]

Headquarters_City=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                   "data-testid":"headquarters"}).text.split(',')[0]

#Headquarters_Country=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
#                   "data-testid":"headquarters"}).text.split(',')[2]

Revenue= soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content", "data-testid":"revenue"}).text


Cmp_size=soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content","data-testid":"employees"}).text

Industry= soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content","data-testid":"revenue"}).text

print(Headquarters_State,Headquarters_City, Cmp_size,Revenue,Industry)

 California. San Jose 10,000+ more than $10B (USD) more than $10B (USD)


### 3. Company Interview

In [105]:
query_3 = "Paypal/interviews"
indeed_search_3 = "https://www.indeed.com/cmp/"
indeed_query_3 = indeed_search_3 + query_3
print(indeed_query_3)
driver.get(indeed_query_3)

https://www.indeed.com/cmp/Paypal/interviews


In [106]:
soup_3 = BeautifulSoup(driver.page_source, 'html.parser')

##### Get Interview difficulty & length

In [107]:
inter_diff=soup_3.find('div',{"class":"cmp-DifficultyCard-result",
                       "data-testid":"cmp-InterviewSummaryCard-result"}).text
print(inter_diff)

Medium


In [21]:
inter_len=soup_3.find('div',{"class":"cmp-DurationCard-text",
                       "data-testid":"cmp-InterviewSummaryCard-result"}).text
print(inter_len)


About a day or two


# Create Function to web scrapping

In [326]:
def web_scrapping(cmp):
    
    headers = ['company','Headquarters_State', 'Headquarters_City', 'revenue', 
               'cmp_size', 'industry','salary_sat', 'inter_diff','inter_len','title', 'salary', 'salary_unit',
              "cmp_rating","benefit_rating"]
    
    cmp1=cmp.replace(" ", "-")
    
    try:
        query1 = cmp1+"/salaries"
        indeed_search = "https://www.indeed.com/cmp/"
        indeed_query1 = indeed_search + query1 +"?job_category=math"
        driver.get(indeed_query1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        ###Get Title/Salary/hourly or year
        salary_span=soup.find_all('span')
        title1=soup.find_all('div', class_="cmp-SalarySummaryTitle")
        
        salary=[]
        salary_unit=[]
        title=[]
        
        i=0
        for salary1 in salary_span:
            try:
                salary2=salary1.text.split(" ")
                if salary2[1]=='per':
                    salary.append(salary2[0])
                    salary_unit.append(salary2[1]+" "+salary2[2])
                    title.append(title1[i].find('a').text)
                    i+=1
            except Exception:
                pass
        ###Get Salary Satisfication rate %
        salary_sat=soup.find('div',class_="cmp-SalarySatisfactionSidebarWidgetPieChart-inside").text
        salary_sat_div=int(salary_sat.replace("%",""))
        salary_sat=[salary_sat_div]*len(salary)
        cmp_rating=[float(soup.find('span', {"aria-hidden":"true", 
                                             "class":"cmp-CompactHeaderCompanyRatings-value"}).text)]*len(salary)
        
        
        query1 = cmp1
        indeed_search = "https://www.indeed.com/cmp/"
        indeed_query1 = indeed_search + query1
        driver.get(indeed_query1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        benefit_rating=[float(soup.find_all('span', {"aria-hidden":"true","class":"css-3xwreb e1wnkr790"})[1].text)]*len(salary)

        ###Get Headquarters/Revenue/company size/ Industry
        query_2 = cmp1
        indeed_search_2 = "https://www.indeed.com/cmp/"
        indeed_query_2 = indeed_search_2 + query_2 + "/about?from=acme-wonder"
        driver.get(indeed_query_2)
        soup_2 = BeautifulSoup(driver.page_source, 'html.parser')
        
 
        try:     
            Headquarters_State=[soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                                  "data-testid":"headquarters"}).text.split(',')[1]]*len(salary)
        except Exception:
            Headquarters_State=np.nan*len(salary)

        try:
            Headquarters_City=[soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                                 "data-testid":"headquarters"}).text.split(',')[0]]*len(salary)
        except Exception:
            Headquarters_City=np.nan*len(salary)
            
        try:    
            revenue= [soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content", 
                                        "data-testid":"revenue"}).text]*len(salary)
        except Exception:
            revenue=np.nan*len(salary) 
            
        try:
            cmp_size=[soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                         "data-testid":"employees"}).text]*len(salary)
        except Exception:
            cmp_size=np.nan*len(salary) 
            
        try:
            industry= [soup_2.find("div",{"class":"cmp-AboutBasicCompanyDetailsWidget-content",
                                          "data-testid":"revenue"}).text]*len(salary)
        except Exception:
            industry=np.nan*len(salary) 
    
        ###Get Interview difficulty & length
        query_3 = cmp1 + "/interviews"
        indeed_search_3 = "https://www.indeed.com/cmp/"
        indeed_query_3 = indeed_search_3 + query_3
        driver.get(indeed_query_3)
        soup_3 = BeautifulSoup(driver.page_source, 'html.parser')
        try:
            inter_diff=[soup_3.find('div',{"class":"cmp-DifficultyCard-result",
                           "data-testid":"cmp-InterviewSummaryCard-result"}).text]*len(salary)
        except Exception:
            inter_diff="none"*len(salary)
            
        try:
            inter_len=[soup_3.find('div',{"class":"cmp-DurationCard-text",
                           "data-testid":"cmp-InterviewSummaryCard-result"}).text]*len(salary)
        except Exception:
            inter_len="none"*len(salary)
        
        
        company=[cmp]*len(salary)
        return dict(zip(headers, [company,Headquarters_State, Headquarters_City, revenue, 
               cmp_size, industry,salary_sat, inter_diff,inter_len,title, salary, salary_unit,cmp_rating,benefit_rating]))


    except Exception:
        return dict(zip(headers, [cmp, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0]))

### For loop-web_scrapping function

In [330]:
#indeed_data = []
change_user=0
sleep_time=0

for link in new_company_list[0][3:]:

    if change_user==0:
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    else:
        driver.quit()
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    change_user+=1
    sleep_time+=1
 
    indeed_data.append(web_scrapping(link))
    if sleep_time % 10==0:
        time.sleep(30+5*random.random())
        
    if sleep_time % 100==0:
        time.sleep(300)
driver.quit()


<ipython-input-330-6ad492444d4d>:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
<ipython-input-330-6ad492444d4d>:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


In [404]:
#indeed_data_1=[]
change_user=1
sleep_time=1

for link in issue_cmp[3:]:

    if change_user==0:
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    else:
        driver.quit()
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    change_user+=1
    sleep_time+=1
 
    indeed_data_1.append(web_scrapping1(link))
    if sleep_time % 50==0:
        time.sleep(30+5*random.random())
        
    if sleep_time % 200==0:
        time.sleep(300)
driver.quit()

<ipython-input-404-1f273e61bc3d>:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


In [407]:
len(indeed_data_1)
c=pd.DataFrame(indeed_data_1[0])
for i in range(1,len(indeed_data_1)):
    try:
        Indeed_Company_Salary_1=pd.concat([Indeed_Company_Salary_1,pd.DataFrame(indeed_data_1[i])])
    except Exception:
        pass

In [360]:
#company_list.remove_all(None)
#print(company_list)

Indeed_Company_Salary=pd.DataFrame(indeed_data[0])
for i in range(1,len(indeed_data)):
    try:
        Indeed_Company_Salary=pd.concat([Indeed_Company_Salary,pd.DataFrame(indeed_data[i])])
    except Exception:
        pass

In [361]:
print(Indeed_Company_Salary.shape)

Indeed_Company_Salary.head(10)

(733, 14)


,company,Headquarters_State,Headquarters_City,revenue,cmp_size,industry,salary_sat,inter_diff,inter_len,title,salary,salary_unit,cmp_rating,benefit_rating
0,Deloitte,EC4A 3HQ\r\nUnited Kingdom,1 New Street Square\r\nLondon,$1B to $5B (USD),"10,000+",$1B to $5B (USD),65.0,Difficult,About a month,Analyst,"$56,670",per year,4.0,3.7
0,UnitedHealth Group,NaN,NaN,NaN,NaN,NaN,60.0,Medium,About two weeks,Analyst,"$39,345",per year,3.7,3.6
1,UnitedHealth Group,NaN,NaN,NaN,NaN,NaN,60.0,Medium,About two weeks,Data Scientist,"$107,613",per year,3.7,3.6
2,UnitedHealth Group,NaN,NaN,NaN,NaN,NaN,60.0,Medium,About two weeks,Senior Analyst,"$47,204",per year,3.7,3.6
3,UnitedHealth Group,NaN,NaN,NaN,NaN,NaN,60.0,Medium,About two weeks,Senior Data Scientist,"$127,991",per year,3.7,3.6
0,PayPal,California.,San Jose,more than $10B (USD),"10,000+",more than $10B (USD),67.0,Medium,About a week,Data Scientist,"$174,553",per year,3.9,4.0
1,PayPal,California.,San Jose,more than $10B (USD),"10,000+",more than $10B (USD),67.0,Medium,About a week,Lead Data Scientist,"$201,747",per year,3.9,4.0
0,General Dynamics Information Technology,Virginia,Fairfax,more than $10B (USD),"10,000+",more than $10B (USD),64.0,Medium,About a day or two,Analyst,"$94,354",per year,3.8,3.8
1,General Dynamics Information Technology,Virginia,Fairfax,more than $10B (USD),"10,000+",more than $10B (USD),64.0,Medium,About a day or two,Biostatistician,"$115,690",per year,3.8,3.8
2,General Dynamics Information Technology,Virginia,Fairfax,more than $10B (USD),"10,000+",more than $10B (USD),64.0,Medium,About a day or two,Data Scientist,"$105,189",per year,3.8,3.8


In [368]:
Indeed_Company_Salary.to_csv('Indeed_Company_Salary.csv', index=False)

In [15]:
driver.quit()

In [415]:
#Indeed_Company_Salary=pd.read_csv("Indeed_Company_Salary_revised.csv")

#Indeed_Company_Salary=pd.concat([Indeed_Company_Salary,Indeed_Company_Salary_1])

#Indeed_Company_Salary.to_csv('Indeed_Company_Salary.csv', index=False)

### Check the company without data, and rerun webscrapping

In [401]:
print(len(indeed_data))
empty=[]
for i,a in enumerate(indeed_data):
    if a==({'company': [],'Headquarters_State': [],
                   'Headquarters_City': [],'revenue': [],'cmp_size': [],'industry': [],
                   'salary_sat': [],'inter_diff': [],'inter_len': [],'title': [],'salary': [],
                   'salary_unit': [],'cmp_rating': [],'benefit_rating': []}):
        empty.append(i)

empty=np.array(empty)+1    
empty

issue_cmp=new_company_list[0][empty]
len(issue_cmp)
print(issue_cmp)


#indeed_data.index({'company': [],'Headquarters_State': [],
#                   'Headquarters_City': [],'revenue': [],'cmp_size': [],'industry': [],
#                   'salary_sat': [],'inter_diff': [],'inter_len': [],'title': [],'salary': [],
#                   'salary_unit': [],'cmp_rating': [],'benefit_rating': []})
#for i in range(13):
#    print(indeed_data[i]['company'])
#new_company_list[0][13]
#indeed_data[3]

1636
13                        Premier Research
20          Logistics Management Institute
24                                  Google
25                       Johnson & Johnson
33                               Capgemini
                       ...                
1612                               Sephora
1613    Institutional Shareholder Services
1614                                  EXOS
1615              Granite Construction Inc
1624                      Raven Industries
Name: 0, Length: 421, dtype: object


### Founded year & industry

In [419]:
company=Indeed_Company_Salary['company'].unique()

In [423]:
driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
query=company[1].replace(" ","-")
indeed_search = "https://www.indeed.com/cmp/"
indeed_query = indeed_search + query
driver.get(indeed_query)
soup = BeautifulSoup(driver.page_source, 'html.parser')


<ipython-input-423-07bba145c6fc>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


In [435]:
def found_industry(cmp):
    try:
        header=["company","founded","industry"]
        company=cmp

        query=cmp.replace(" ","-")
        indeed_search = "https://www.indeed.com/cmp/"
        indeed_query = indeed_search + query
        driver.get(indeed_query)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        founded=soup.find('li', {"data-testid":"companyInfo-founded", "class":"css-v4e08k eu4oa1w0"}).text
        founded=int(founded.replace("Founded",""))

        industry=soup.find('li', {"data-testid":"companyInfo-industry", "class":"css-v4e08k eu4oa1w0"}).text
        industry=industry.replace("Industry","")


        return dict(zip(header, [company,founded, industry]))


    except Exception:
        return dict(zip(headers, [cmp, 0, 0]))

In [448]:
change_user=0
sleep_time=0
indeed_data_2=[]

for link in company[0:]:

    if change_user==0:
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    else:
        driver.quit()
        options = Options()
        ua = UserAgent()
        user_agent = ua.random
        options.add_argument(f'user-agent={user_agent}')
        driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)

    change_user+=1
    sleep_time+=1
 
    indeed_data_2.append(found_industry(link))
    if sleep_time % 50==0:
        time.sleep(20+5*random.random())
        
    if sleep_time % 200==0:
        time.sleep(300)
driver.quit()



<ipython-input-448-dc803e5a3e25>:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)
<ipython-input-448-dc803e5a3e25>:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path=chromedriver)


In [452]:
indeed_data_2[0]

{'company': 'Deloitte', 'founded': 1850, 'industry': 'Information Technology'}

In [466]:



industry_found=pd.DataFrame(indeed_data_2[0],index=[0])

for i in range(1,len(indeed_data_2)):
    try:
        industry_found=pd.concat([industry_found,pd.DataFrame(indeed_data_2[i],index=[i])])
    except Exception:
        pass

industry_found.to_csv('industry_found.csv', index=False)

In [467]:
industry_found

,company,founded,industry,Headquarters_State,Headquarters_City
0,Deloitte,1850.0,Information Technology,NaN,NaN
1,UnitedHealth Group,1977.0,Healthcare,NaN,NaN
2,PayPal,1998.0,Financial Services,NaN,NaN
3,General Dynamics Information Technology,1952.0,Aerospace & Defense,NaN,NaN
4,Centene,1984.0,Healthcare and insurance,NaN,NaN
...,...,...,...,...,...
437,The Aerospace Corporation,1960.0,Aerospace & Defense,NaN,NaN
438,"Qualcomm Technologies, Inc.",1985.0,Information Technology,NaN,NaN
439,Qualcomm Canada ULC,1985.0,Information Technology,NaN,NaN
440,"Guidewire Software, Inc.",2001.0,Insurance,NaN,NaN


In [465]:
pd.concat([industry_found,pd.DataFrame(indeed_data_2[1],index=[1])])

,company,founded,industry
0,Deloitte,1850,Information Technology
1,UnitedHealth Group,1977,Healthcare
